# Running Tune experiments with Dragonfly

This example demonstrates the usage of Dragonfly with Ray Tune via `DragonflySearch`.

It also also shows that a scheduler can be used with `DragonflySearch`, e.g. `AsyncHyperBandScheduler`.

Background information:
- [Dragonfly website](https://dragonfly-opt.readthedocs.io/)

Necessary requirements:
- `pip install ray[tune]`
- `pip install dragonfly-opt==0.1.6`

In [ ]:
# !pip install ray[tune]
!pip install dragonfly-opt==0.1.6

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [ ]:
import numpy as np
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch

Let's start by defining a optimization problem. Suppose we want to figure out the proportions of water and several salts to add to an ionic solution with the goal of maximizing it's ability to conduct electricity.

The objective here is explicit for demonstration, yet in practice they often come out of a black-box (e.g. a physical device measuring conductivity, or reporting the results of a long-running ML experiment).

We artificially sleep for a bit (`0.02` seconds) to simulate a more typical experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune relative proportions of 4 ingredients-- these proportions should be considered as hyperparameters.

Our ``objective`` function will take a Tune ``config``, evaluates the `conductivity` of our experiment in a training loop,
and uses `tune.report` to report the `conductivity` back to Tune.

In [ ]:
def objective(config):
    """
    Simplistic model of electrical conductivity with added Gaussian noise to simulate experimental noise.
    """
    for i in range(config["iterations"]):
        vol1 = config["LiNO3_vol"]  # LiNO3
        vol2 = config["Li2SO4_vol"]  # Li2SO4
        vol3 = config["NaClO4_vol"]  # NaClO4
        vol4 = 10 - (vol1 + vol2 + vol3)  # Water
        conductivity = vol1 + 0.1 * (vol2 + vol3) ** 2 + 2.3 * vol4 * (vol1 ** 1.5)
        conductivity += np.random.normal() * 0.01
        tune.report(timesteps_total=i, objective=conductivity)
        time.sleep(0.02)

In [ ]:
ray.init(configure_logging=False)

Now we define the search algorithm from `DragonflySearch`  with `optimizer` and `domain` arguments specified in a common way.

In [ ]:
algo = DragonflySearch(
    optimizer="bandit",
    domain="euclidean",
)

We also use `ConcurrencyLimiter` to constrain to 4 concurrent trials. We include a scheduler, `AsyncHyperBandScheduler`, to demonstrate the modularity of Ray Tune. 

In [ ]:
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
num_samples = 100

In [ ]:
# Reducing samples for smoke tests
num_samples = 10

Now we run the experiment:

In [ ]:
analysis = tune.run(
    objective,
    metric="objective",
    mode="max",
    name="dragonfly_search",
    search_alg=algo,
    scheduler=scheduler,
    num_samples=num_samples,
    config={
        "iterations": 100,
        "LiNO3_vol": tune.uniform(0, 7),
        "Li2SO4_vol": tune.uniform(0, 7),
        "NaClO4_vol": tune.uniform(0, 7)
    },
)

Below are the recommended relative proportions of water and each salt found to maximize conductivity in the ionic solution (according to the simple model):

In [ ]:
print("Best hyperparameters found: ", analysis.best_config)

In [ ]:
ray.shutdown()